<a href="https://colab.research.google.com/github/vaasu14/pyspark/blob/main/starting_with_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
!pwd

/content


In [3]:
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1//spark-3.1.1-bin-hadoop3.2.tgz  

In [4]:
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [5]:
!pwd

/content


In [6]:
cd /content/spark-3.1.1-bin-hadoop3.2

/content/spark-3.1.1-bin-hadoop3.2


In [7]:
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [9]:
import findspark
findspark.init()

In [10]:
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark').getOrCreate()

In [12]:
cd /content/spark-3.1.1-bin-hadoop3.2

/content/spark-3.1.1-bin-hadoop3.2


In [13]:
from google.colab import files
df=files.upload()

Saving weatherHistory.csv to weatherHistory.csv


In [15]:
df=spark.read.csv('/content/spark-3.1.1-bin-hadoop3.2/weatherHistory.csv',header=True,inferSchema=True)

In [16]:
df.head(5)

[Row(Formatted Date='2006-04-01 00:00:00.000 +0200', Summary='Partly Cloudy', Precip Type='rain', Temperature (C)=9.472222222222221, Apparent Temperature (C)=7.3888888888888875, Humidity=0.89, Wind Speed (km/h)=14.1197, Wind Bearing (degrees)=251.0, Visibility (km)=15.826300000000002, Loud Cover=0.0, Pressure (millibars)=1015.13, Daily Summary='Partly cloudy throughout the day.'),
 Row(Formatted Date='2006-04-01 01:00:00.000 +0200', Summary='Partly Cloudy', Precip Type='rain', Temperature (C)=9.355555555555558, Apparent Temperature (C)=7.227777777777776, Humidity=0.86, Wind Speed (km/h)=14.2646, Wind Bearing (degrees)=259.0, Visibility (km)=15.826300000000002, Loud Cover=0.0, Pressure (millibars)=1015.63, Daily Summary='Partly cloudy throughout the day.'),
 Row(Formatted Date='2006-04-01 02:00:00.000 +0200', Summary='Mostly Cloudy', Precip Type='rain', Temperature (C)=9.377777777777778, Apparent Temperature (C)=9.377777777777778, Humidity=0.89, Wind Speed (km/h)=3.9284000000000003, Win

In [17]:
df.printSchema()

root
 |-- Formatted Date: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Precip Type: string (nullable = true)
 |-- Temperature (C): double (nullable = true)
 |-- Apparent Temperature (C): double (nullable = true)
 |-- Humidity: double (nullable = true)
 |-- Wind Speed (km/h): double (nullable = true)
 |-- Wind Bearing (degrees): double (nullable = true)
 |-- Visibility (km): double (nullable = true)
 |-- Loud Cover: double (nullable = true)
 |-- Pressure (millibars): double (nullable = true)
 |-- Daily Summary: string (nullable = true)



In [18]:
df.describe().show()

+-------+--------------------+--------------------+-----------+-------------------+------------------------+------------------+------------------+----------------------+------------------+----------+--------------------+--------------------+
|summary|      Formatted Date|             Summary|Precip Type|    Temperature (C)|Apparent Temperature (C)|          Humidity| Wind Speed (km/h)|Wind Bearing (degrees)|   Visibility (km)|Loud Cover|Pressure (millibars)|       Daily Summary|
+-------+--------------------+--------------------+-----------+-------------------+------------------------+------------------+------------------+----------------------+------------------+----------+--------------------+--------------------+
|  count|               96453|               96453|      96453|              96453|                   96453|             96453|             96453|                 96453|             96453|     96453|               96453|               96453|
|   mean|                null|  

In [20]:
from pyspark.sql.functions import corr
df.select(corr('Humidity','Temperature (C)')).show()

+-------------------------------+
|corr(Humidity, Temperature (C))|
+-------------------------------+
|            -0.6322546750278134|
+-------------------------------+



In [21]:
df.select('Summary').distinct().show()

+--------------------+
|             Summary|
+--------------------+
|              Breezy|
|Humid and Mostly ...|
|  Windy and Overcast|
|               Foggy|
|Humid and Partly ...|
|     Windy and Foggy|
|Breezy and Partly...|
|                 Dry|
|             Drizzle|
|      Breezy and Dry|
|       Partly Cloudy|
|               Clear|
|       Windy and Dry|
|       Mostly Cloudy|
|    Breezy and Foggy|
| Breezy and Overcast|
|Dangerously Windy...|
|          Light Rain|
|Breezy and Mostly...|
|Dry and Mostly Cl...|
+--------------------+
only showing top 20 rows



In [22]:
df.columns

['Formatted Date',
 'Summary',
 'Precip Type',
 'Temperature (C)',
 'Apparent Temperature (C)',
 'Humidity',
 'Wind Speed (km/h)',
 'Wind Bearing (degrees)',
 'Visibility (km)',
 'Loud Cover',
 'Pressure (millibars)',
 'Daily Summary']

In [23]:
from pyspark.ml.regression import LinearRegression

In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [25]:
assembler= VectorAssembler(inputCols=['Humidity','Wind Speed (km/h)','Loud Cover','Pressure (millibars)',
                                      'Wind Bearing (degrees)','Apparent Temperature (C)','Visibility (km)'],outputCol='features')

In [26]:
output=assembler.transform(df)

In [27]:
output.printSchema()

root
 |-- Formatted Date: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Precip Type: string (nullable = true)
 |-- Temperature (C): double (nullable = true)
 |-- Apparent Temperature (C): double (nullable = true)
 |-- Humidity: double (nullable = true)
 |-- Wind Speed (km/h): double (nullable = true)
 |-- Wind Bearing (degrees): double (nullable = true)
 |-- Visibility (km): double (nullable = true)
 |-- Loud Cover: double (nullable = true)
 |-- Pressure (millibars): double (nullable = true)
 |-- Daily Summary: string (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
final_data=output.select('features','Temperature (C)')

In [29]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [30]:
lr=LinearRegression(labelCol='Temperature (C)')

In [34]:
lr_model=lr.fit(train_data)

In [35]:
test_results=lr_model.evaluate(test_data)

In [36]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.7897372877011994|
|  -2.319957478452939|
|  0.7029078672809064|
| -0.8556577870613422|
|  1.1194014821080778|
| -0.2017947345586748|
| 0.09847293355140074|
| -0.8412206748683699|
|-0.36822704627151914|
| -1.2811816818278912|
| -1.1347962985926525|
|-0.08141735433788355|
| -3.1794400173217396|
| -1.1120370537919229|
| 0.42184359225515067|
| 0.29456614260475256|
| 0.40071857600040905|
|  1.7334518852526895|
| -2.2646487060994964|
|-0.15524801687148937|
+--------------------+
only showing top 20 rows



In [37]:
test_results.rootMeanSquaredError

0.941249657233806

In [38]:
test_results.r2

0.9902731171846357

In [49]:
unlabelled_data=test_data.select('features')

In [50]:
unlabelled_data.show()

+--------------------+
|            features|
+--------------------+
|(7,[0,3,5],[0.6,1...|
|(7,[0,3,5],[0.92,...|
|(7,[0,3,5],[0.96,...|
|(7,[0,3,5],[1.0,1...|
|(7,[0,5,6],[0.5,2...|
|(7,[0,5,6],[0.77,...|
|(7,[0,5,6],[0.78,...|
|(7,[0,5,6],[0.79,...|
|(7,[0,5,6],[0.8,1...|
|(7,[0,5,6],[0.92,...|
|(7,[0,5,6],[0.96,...|
|(7,[0,5,6],[1.0,1...|
|[0.0,4.4275,0.0,1...|
|[0.0,6.44,0.0,101...|
|[0.0,11.270000000...|
|[0.0,11.270000000...|
|[0.0,20.93,0.0,10...|
|[0.0,22.540000000...|
|[0.14,20.93,0.0,1...|
|[0.15,5.2969,0.0,...|
+--------------------+
only showing top 20 rows



In [51]:
prediction=lr_model.transform(unlabelled_data)

In [53]:
df.select(test_data['Temperature (c)']).show()

+------------------+
|   Temperature (c)|
+------------------+
| 9.472222222222221|
| 9.355555555555558|
| 9.377777777777778|
|  8.28888888888889|
| 8.755555555555553|
| 9.222222222222221|
| 7.733333333333334|
|  8.77222222222222|
| 10.82222222222222|
| 13.77222222222222|
|16.016666666666666|
|17.144444444444446|
|17.800000000000004|
|17.333333333333332|
| 18.87777777777778|
|18.911111111111115|
| 15.38888888888889|
|15.550000000000002|
|14.255555555555553|
|13.144444444444442|
+------------------+
only showing top 20 rows



In [42]:
prediction.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(7,[0,3,5],[0.6,1...|  8.567515065478977|
|(7,[0,3,5],[0.92,...| -5.457820299324839|
|(7,[0,3,5],[0.96,...| 14.852647688274649|
|(7,[0,3,5],[1.0,1...| 3.7001022315057854|
|(7,[0,5,6],[0.5,2...| 24.991709629003033|
|(7,[0,5,6],[0.77,...| 12.951794734558677|
|(7,[0,5,6],[0.78,...|   14.9015270664486|
|(7,[0,5,6],[0.79,...|  8.063442897090592|
|(7,[0,5,6],[0.8,1...|  11.47933815738263|
|(7,[0,5,6],[0.92,...| 3.5034039040501135|
|(7,[0,5,6],[0.96,...|   3.91257407637043|
|(7,[0,5,6],[1.0,1...| 11.242528465448997|
|[0.0,4.4275,0.0,1...| 2.0683289062106285|
|[0.0,6.44,0.0,101...|-14.443518501763633|
|[0.0,11.270000000...| -16.53295470336626|
|[0.0,11.270000000...|-14.183455031493642|
|[0.0,20.93,0.0,10...| 0.7103925351107021|
|[0.0,22.540000000...| -13.95567410747491|
|[0.14,20.93,0.0,1...| 17.264648706099496|
|[0.15,5.2969,0.0,...|  21.36080357242705|
+----------

PYSPARK MLlib